In [1]:
import ccxt
import pandas as pd

# 定義交易對
symbol = 'PENGU/USDT'  # 假設 PENGU 是以 USDT 計價的
listing_time = '2023-01-01T00:00:00Z'  # 假設上幣日期
since = None

# 獲取幣種的上幣時間
def get_since(exchange):
    return exchange.parse8601(listing_time)

# 獲取歷史交易數據
def fetch_ohlcv(exchange, symbol):
    since = get_since(exchange)
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1h', since=since)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.drop(columns=['timestamp'], inplace=True)
    return df

# 初始化交易所
exchanges = {
    'binance': ccxt.binance(),
    #'okx': ccxt.okx(),
    # 'bybit': ccxt.bybit(),
}

# 獲取數據並保存為 CSV
for exchange_name, exchange in exchanges.items():
    try:
        df = fetch_ohlcv(exchange, symbol)
        csv_filename = f'{exchange_name}_pengu_ohlcv_data.csv'
        df.to_csv(csv_filename, index=False)
        print(f"Data saved to {csv_filename}")
    except Exception as e:
        print(f"Error fetching data from {exchange_name}: {e}")

/Users/andyouyang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Data saved to binance_pengu_ohlcv_data.csv


In [2]:
import requests
import pandas as pd

# 定義 OKX API URL
base_url = 'https://www.okx.com/api/v5'

# 定義交易對和幣種上幣時間
symbol = 'PENGU-USDT'  # 假設 PENGU 是以 USDT 計價的
listing_time = '2023-01-01T00:00:00Z'  # 假設上幣日期

# 將上幣時間轉換為毫秒時間戳
since = int(pd.to_datetime(listing_time).timestamp() * 1000)

# 獲取歷史 OHLCV 數據
def fetch_ohlcv(symbol, since):
    url = f'{base_url}/market/candles?instId={symbol}&bar=1H&start={since}'
    response = requests.get(url)

    # 檢查請求是否成功
    if response.status_code == 200:
        data = response.json()
        if data['code'] == '0':
            return data['data']
        else:
            print(f"Error: {data['msg']}")
            return None
    else:
        print(f"HTTP Error: {response.status_code}")
        return None

# 獲取數據並保存為 CSV
ohlcv_data = fetch_ohlcv(symbol, since)
if ohlcv_data:
    # 將數據轉換為 DataFrame
    df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'count'])
    
    # 將 timestamp 轉換為可讀的日期時間格式
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')

    # 刪除不需要的列
    df.drop(columns=['timestamp', 'close_time', 'quote_volume', 'count'], inplace=True)

    # 顯示 DataFrame
    print(df)

    # 將 DataFrame 保存為 CSV 文件
    df.to_csv('okx_pengu_ohlcv_data.csv', index=False)
    print("Data saved to okx_pengu_ohlcv_data.csv")

        open      high       low     close     volume                date
0    0.02918   0.02921  0.028044  0.028129   45035381 2024-12-20 01:00:00
1   0.028625  0.029849  0.028348  0.029169   70200072 2024-12-20 00:00:00
2   0.029441  0.029468    0.0286  0.028624   23219171 2024-12-19 23:00:00
3   0.029006  0.029449  0.028561  0.029449   47553100 2024-12-19 22:00:00
4   0.028211  0.029125  0.028019     0.029   45144315 2024-12-19 21:00:00
5   0.028076  0.028426  0.026861  0.028203   54917218 2024-12-19 20:00:00
6   0.028778  0.028812  0.027317   0.02807   37163915 2024-12-19 19:00:00
7   0.030233  0.030537  0.028422  0.028769   69779275 2024-12-19 18:00:00
8   0.031051  0.031086  0.029809  0.030258   65480608 2024-12-19 17:00:00
9   0.030659  0.031639  0.030365  0.031035   71647659 2024-12-19 16:00:00
10  0.030206  0.031193  0.029206  0.030677   98940707 2024-12-19 15:00:00
11  0.029711  0.030491  0.029399  0.030212   64817469 2024-12-19 14:00:00
12  0.030575  0.030759  0.029132  0.02

/var/folders/ks/h0cwv1cn5t92_dn23lyd5_kw0000gn/T/ipykernel_3080/3414888606.py:38: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['date'] = pd.to_datetime(df['timestamp'], unit='ms')


In [3]:
import ccxt
import csv
from pprint import pprint

# 创建 Bybit 实例，替换为您的 API 密钥和秘密
bybit = ccxt.bybit()

# 加载市场信息
markets = bybit.load_markets()

# 获取 OHLCV 数据
symbol = 'PENGU/USDT'  # 确保符号格式正确
try:
    ohlcv = bybit.fetch_ohlcv(symbol, timeframe='1h', limit=200)
    print(ohlcv)

    # 保存为 CSV 文件
    with open('bybit_pengu_ohlcv_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # 写入表头
        writer.writerow(['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        # 写入数据
        writer.writerows(ohlcv)

    print("OHLCV 数据已成功保存为 ohlcv_data.csv")
except Exception as e:
    print(f"Error fetching OHLCV data: {e}")


[[1734444000000, 0.0034, 0.05851, 0.0034, 0.0393, 231574687.0], [1734447600000, 0.0393, 0.04217, 0.03601, 0.03661, 139399196.0], [1734451200000, 0.03661, 0.03936, 0.03133, 0.03273, 236513212.0], [1734454800000, 0.03273, 0.03299, 0.02884, 0.03105, 154654512.0], [1734458400000, 0.03105, 0.035, 0.0298, 0.03448, 147092363.0], [1734462000000, 0.03448, 0.03534, 0.03132, 0.03174, 99864040.0], [1734465600000, 0.03174, 0.03188, 0.03027, 0.03097, 61815222.0], [1734469200000, 0.03097, 0.0312, 0.02834, 0.02971, 59806291.0], [1734472800000, 0.02971, 0.031, 0.02685, 0.02775, 60102897.0], [1734476400000, 0.02775, 0.02903, 0.02711, 0.02875, 45539869.0], [1734480000000, 0.02875, 0.03066, 0.02693, 0.02722, 60085972.0], [1734483600000, 0.02722, 0.02793, 0.0265, 0.0271, 46024580.0], [1734487200000, 0.0271, 0.03188, 0.02662, 0.03106, 137488974.0], [1734490800000, 0.03106, 0.0325, 0.03027, 0.03099, 88509666.0], [1734494400000, 0.03099, 0.0319, 0.02927, 0.03049, 41907558.0], [1734498000000, 0.03049, 0.03122,

In [5]:
import ccxt
import csv
from pprint import pprint

# 创建 mexc 实例，替换为您的 API 密钥和秘密
mexc = ccxt.mexc()

# 加载市场信息
markets = mexc.load_markets()

# 获取 OHLCV 数据
symbol = 'PENGU/USDT'  # 确保符号格式正确
try:
    ohlcv = mexc.fetch_ohlcv(symbol, timeframe='1h', limit=200)
    print(ohlcv)

    # 保存为 CSV 文件
    with open('mexc_pengu_ohlcv_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # 写入表头
        writer.writerow(['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        # 写入数据
        writer.writerows(ohlcv)

    print("OHLCV 数据已成功保存为 ohlcv_data.csv")
except Exception as e:
    print(f"Error fetching OHLCV data: {e}")


[[1734440400000, 0.01, 0.099, 0.01, 0.05656, 63128610.49], [1734444000000, 0.05656, 0.0567, 0.04209, 0.04216, 332105440.05], [1734447600000, 0.04216, 0.04225, 0.03401, 0.03677, 155770662.9], [1734451200000, 0.03677, 0.03961, 0.03088, 0.03154, 256899285.19], [1734454800000, 0.03154, 0.03352, 0.02621, 0.03104, 167137293.71], [1734458400000, 0.03104, 0.035, 0.02991, 0.03447, 140117790.56], [1734462000000, 0.03447, 0.0352, 0.0314, 0.03181, 81087336.12], [1734465600000, 0.03181, 0.03192, 0.03055, 0.031, 46143202.55], [1734469200000, 0.031, 0.0312, 0.02846, 0.02969, 48758896.7], [1734472800000, 0.02969, 0.03095, 0.0268, 0.02772, 69013105.17], [1734476400000, 0.02772, 0.02891, 0.027, 0.0287, 39644368.95], [1734480000000, 0.0287, 0.03, 0.02689, 0.02714, 52849976.5], [1734483600000, 0.02714, 0.02778, 0.02628, 0.02708, 50039857.47], [1734487200000, 0.02708, 0.032, 0.02666, 0.03094, 136425407.01], [1734490800000, 0.03094, 0.03404, 0.0303, 0.03105, 57962709.79], [1734494400000, 0.03105, 0.03178, 0

In [7]:
import ccxt
import csv
from pprint import pprint

# 创建 mexc 实例，替换为您的 API 密钥和秘密
lbank = ccxt.lbank()

# 加载市场信息
markets = lbank.load_markets()

# 获取 OHLCV 数据
symbol = 'PENGU/USDT'  # 确保符号格式正确
try:
    ohlcv = lbank.fetch_ohlcv(symbol, timeframe='1h')
    print(ohlcv)

    # 保存为 CSV 文件
    with open('mexc_pengu_ohlcv_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # 写入表头
        writer.writerow(['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        # 写入数据
        writer.writerows(ohlcv)

    print("OHLCV 数据已成功保存为 ohlcv_data.csv")
except Exception as e:
    print(f"Error fetching OHLCV data: {e}")


NetworkError: lbank GET https://lbkperp.lbank.com/cfd/openApi/v1/pub/instrument?productGroup=SwapU